In [1]:
import numpy as np
import tensorflow as tf

In [2]:
NUM_DIGITS = 10

# Represent each input by an array of its binary digits.
def binary_encode(i, NUM_DIGITS):
    return np.array([i >> d & 1 for d in range(NUM_DIGITS)])

print(binary_encode(1,100))

[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [3]:
# One-hot encode the desired outputs: [number, "fizz", "buzz", "fizzbuzz"]
def fizz_buzz_encode(i):
    if   i % 15 == 0: return np.array([0, 0, 0, 1])
    elif i % 5  == 0: return np.array([0, 0, 1, 0])
    elif i % 3  == 0: return np.array([0, 1, 0, 0])
    else:             return np.array([1, 0, 0, 0])

In [4]:
# Our goal is to produce fizzbuzz for the numbers 1 to 100. So it would be
# unfair to include these in our training data. Accordingly, the training data
# corresponds to the numbers 101 to (2 ** NUM_DIGITS - 1).
trX = np.array([binary_encode(i, NUM_DIGITS) for i in range(101, 2 ** NUM_DIGITS)])
trY = np.array([fizz_buzz_encode(i)          for i in range(101, 2 ** NUM_DIGITS)])

In [5]:
# We'll want to randomly initialize weights.
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

In [6]:
# Our model is a standard 1-hidden-layer multi-layer-perceptron with ReLU
# activation. The softmax (which turns arbitrary real-valued outputs into
# probabilities) gets applied in the cost function.
def model(X, w_h, w_o):
    h = tf.nn.relu(tf.matmul(X, w_h))
    return tf.matmul(h, w_o)

In [7]:
# Our variables. The input has width NUM_DIGITS, and the output has width 4.
X = tf.placeholder("float", [None, NUM_DIGITS])
Y = tf.placeholder("float", [None, 4])

In [8]:
# How many units in the hidden layer.
NUM_HIDDEN = 100

In [9]:
# Initialize the weights.
w_h = init_weights([NUM_DIGITS, NUM_HIDDEN])
w_o = init_weights([NUM_HIDDEN, 4])

In [10]:
# Predict y given x using the model.
py_x = model(X, w_h, w_o)

In [18]:
# We'll train our model by minimizing a cost function.
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=py_x, labels=Y))
train_op = tf.train.GradientDescentOptimizer(0.05).minimize(cost)

In [19]:
# And we'll make predictions by choosing the largest output.
predict_op = tf.argmax(py_x, 1)

In [20]:
# Finally, we need a way to turn a prediction (and an original number)
# into a fizz buzz output
def fizz_buzz(i, prediction):
    return [str(i), "fizz", "buzz", "fizzbuzz"][prediction]

In [26]:
BATCH_SIZE = 64

In [28]:
with tf.Session() as sess:
    tf.initialize_all_variables().run()

    for epoch in range(4000):
        # Shuffle the data before each training iteration.
        p = np.random.permutation(range(len(trX)))
        trX, trY = trX[p], trY[p]

        # Train in batches of 128 inputs.
        for start in range(0, len(trX), BATCH_SIZE):
            end = start + BATCH_SIZE
            sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end]})

        # And print the current accuracy on the training data.
        print(epoch, np.mean(np.argmax(trY, axis=1) ==
                             sess.run(predict_op, feed_dict={X: trX, Y: trY})))

    # And now for some fizz buzz
    numbers = np.arange(1, 101)
    teX = np.transpose(binary_encode(numbers, NUM_DIGITS))
    teY = sess.run(predict_op, feed_dict={X: teX})
    output = np.vectorize(fizz_buzz)(numbers, teY)

    print(output)

0 0.5341278439869989
1 0.5341278439869989
2 0.5341278439869989
3 0.5341278439869989
4 0.5341278439869989
5 0.5341278439869989
6 0.5341278439869989
7 0.5341278439869989
8 0.5341278439869989
9 0.5341278439869989
10 0.5341278439869989
11 0.5341278439869989
12 0.5341278439869989
13 0.5341278439869989
14 0.5341278439869989
15 0.5341278439869989
16 0.5341278439869989
17 0.5341278439869989
18 0.5341278439869989
19 0.5341278439869989
20 0.5341278439869989
21 0.5341278439869989
22 0.5341278439869989
23 0.5341278439869989
24 0.5341278439869989
25 0.5341278439869989
26 0.5341278439869989
27 0.5341278439869989
28 0.5341278439869989
29 0.5341278439869989
30 0.5341278439869989
31 0.5341278439869989
32 0.5341278439869989
33 0.5341278439869989
34 0.5341278439869989
35 0.5341278439869989
36 0.5341278439869989
37 0.5341278439869989
38 0.5341278439869989
39 0.5341278439869989
40 0.5341278439869989
41 0.5341278439869989
42 0.5341278439869989
43 0.5341278439869989
44 0.5341278439869989
45 0.534127843986998

361 0.5341278439869989
362 0.5341278439869989
363 0.5341278439869989
364 0.5341278439869989
365 0.5341278439869989
366 0.5341278439869989
367 0.5341278439869989
368 0.5341278439869989
369 0.5341278439869989
370 0.5341278439869989
371 0.5341278439869989
372 0.5341278439869989
373 0.5341278439869989
374 0.5341278439869989
375 0.5341278439869989
376 0.5341278439869989
377 0.5341278439869989
378 0.5341278439869989
379 0.5341278439869989
380 0.5341278439869989
381 0.5341278439869989
382 0.5341278439869989
383 0.5341278439869989
384 0.5341278439869989
385 0.5341278439869989
386 0.5341278439869989
387 0.5341278439869989
388 0.5341278439869989
389 0.5341278439869989
390 0.5341278439869989
391 0.5341278439869989
392 0.5341278439869989
393 0.5341278439869989
394 0.5341278439869989
395 0.5341278439869989
396 0.5341278439869989
397 0.5341278439869989
398 0.5341278439869989
399 0.5341278439869989
400 0.5341278439869989
401 0.5341278439869989
402 0.5341278439869989
403 0.5341278439869989
404 0.53412

726 0.6143011917659805
727 0.6034669555796316
728 0.6251354279523293
729 0.6240520043336945
730 0.6186348862405201
731 0.6229685807150596
732 0.6229685807150596
733 0.6240520043336945
734 0.6240520043336945
735 0.6110509209100758
736 0.6251354279523293
737 0.628385698808234
738 0.6294691224268689
739 0.6273022751895991
740 0.6240520043336945
741 0.6229685807150596
742 0.6294691224268689
743 0.6197183098591549
744 0.628385698808234
745 0.6273022751895991
746 0.6197183098591549
747 0.6251354279523293
748 0.6338028169014085
749 0.6218851570964247
750 0.6262188515709642
751 0.628385698808234
752 0.6186348862405201
753 0.6348862405200434
754 0.638136511375948
755 0.6392199349945829
756 0.6240520043336945
757 0.628385698808234
758 0.638136511375948
759 0.6327193932827736
760 0.6305525460455038
761 0.628385698808234
762 0.6446370530877573
763 0.6435536294691224
764 0.6338028169014085
765 0.6392199349945829
766 0.6229685807150596
767 0.638136511375948
768 0.6435536294691224
769 0.6316359696641

1082 0.8353196099674973
1083 0.8179848320693391
1084 0.789815817984832
1085 0.8244853737811484
1086 0.8147345612134345
1087 0.8504875406283857
1088 0.8374864572047671
1089 0.8147345612134345
1090 0.8255687973997833
1091 0.8461538461538461
1092 0.8244853737811484
1093 0.8483206933911159
1094 0.8158179848320694
1095 0.8450704225352113
1096 0.8212351029252438
1097 0.8320693391115926
1098 0.8429035752979415
1099 0.8504875406283857
1100 0.8223185265438786
1101 0.8559046587215602
1102 0.856988082340195
1103 0.8591549295774648
1104 0.856988082340195
1105 0.8353196099674973
1106 0.789815817984832
1107 0.8364030335861322
1108 0.8450704225352113
1109 0.8494041170097508
1110 0.8721560130010835
1111 0.8331527627302275
1112 0.8559046587215602
1113 0.8364030335861322
1114 0.8537378114842904
1115 0.8656554712892741
1116 0.8494041170097508
1117 0.8255687973997833
1118 0.8201516793066089
1119 0.8526543878656555
1120 0.847237269772481
1121 0.8169014084507042
1122 0.8602383531960996
1123 0.85590465872156

1434 0.9501625135427952
1435 0.9404117009750813
1436 0.9490790899241603
1437 0.9555796316359697
1438 0.9447453954496208
1439 0.942578548212351
1440 0.9219934994582882
1441 0.9534127843986999
1442 0.9447453954496208
1443 0.9512459371614301
1444 0.9458288190682557
1445 0.9577464788732394
1446 0.9512459371614301
1447 0.9555796316359697
1448 0.9436619718309859
1449 0.952329360780065
1450 0.9534127843986999
1451 0.9577464788732394
1452 0.9588299024918743
1453 0.9544962080173348
1454 0.9577464788732394
1455 0.9133261105092091
1456 0.9349945828819068
1457 0.9599133261105092
1458 0.9566630552546046
1459 0.9371614301191766
1460 0.9209100758396533
1461 0.9501625135427952
1462 0.9512459371614301
1463 0.9187432286023836
1464 0.9328277356446371
1465 0.9371614301191766
1466 0.952329360780065
1467 0.9544962080173348
1468 0.9544962080173348
1469 0.9599133261105092
1470 0.9631635969664138
1471 0.9458288190682557
1472 0.962080173347779
1473 0.9653304442036836
1474 0.9544962080173348
1475 0.9490790899241

1777 0.9794149512459371
1778 0.9891657638136512
1779 0.9750812567713976
1780 0.9869989165763814
1781 0.9880823401950163
1782 0.9826652221018418
1783 0.9761646803900325
1784 0.9880823401950163
1785 0.9869989165763814
1786 0.9783315276273022
1787 0.9848320693391116
1788 0.9869989165763814
1789 0.980498374864572
1790 0.9848320693391116
1791 0.9880823401950163
1792 0.9729144095341279
1793 0.9783315276273022
1794 0.9880823401950163
1795 0.9869989165763814
1796 0.9794149512459371
1797 0.9859154929577465
1798 0.9859154929577465
1799 0.9859154929577465
1800 0.9761646803900325
1801 0.9761646803900325
1802 0.9826652221018418
1803 0.9869989165763814
1804 0.9869989165763814
1805 0.9902491874322861
1806 0.9826652221018418
1807 0.9891657638136512
1808 0.9848320693391116
1809 0.9696641386782232
1810 0.9869989165763814
1811 0.9891657638136512
1812 0.9880823401950163
1813 0.9891657638136512
1814 0.9891657638136512
1815 0.9880823401950163
1816 0.9815817984832069
1817 0.9815817984832069
1818 0.9859154929

2129 0.9859154929577465
2130 0.9924160346695557
2131 0.9924160346695557
2132 0.9869989165763814
2133 0.9967497291440953
2134 0.9956663055254604
2135 0.9956663055254604
2136 0.9924160346695557
2137 0.9653304442036836
2138 0.9956663055254604
2139 0.9924160346695557
2140 0.9956663055254604
2141 0.9945828819068255
2142 0.9956663055254604
2143 0.9891657638136512
2144 0.9956663055254604
2145 0.9934994582881906
2146 0.9934994582881906
2147 0.9924160346695557
2148 0.9945828819068255
2149 0.9934994582881906
2150 0.9956663055254604
2151 0.9924160346695557
2152 0.9934994582881906
2153 0.991332611050921
2154 0.9934994582881906
2155 0.9934994582881906
2156 0.9967497291440953
2157 0.9967497291440953
2158 0.9924160346695557
2159 0.991332611050921
2160 0.9924160346695557
2161 0.9945828819068255
2162 0.9945828819068255
2163 0.991332611050921
2164 0.9924160346695557
2165 0.9924160346695557
2166 0.9934994582881906
2167 0.9924160346695557
2168 0.9967497291440953
2169 0.9967497291440953
2170 0.993499458288

2476 0.9956663055254604
2477 0.9989165763813651
2478 0.9978331527627302
2479 0.9945828819068255
2480 0.9989165763813651
2481 0.9978331527627302
2482 0.9989165763813651
2483 0.9989165763813651
2484 0.9945828819068255
2485 0.9989165763813651
2486 0.9978331527627302
2487 0.9989165763813651
2488 0.9989165763813651
2489 0.9989165763813651
2490 0.9989165763813651
2491 0.9956663055254604
2492 0.9934994582881906
2493 0.9945828819068255
2494 0.9989165763813651
2495 0.9989165763813651
2496 0.9989165763813651
2497 0.9978331527627302
2498 0.9989165763813651
2499 0.9967497291440953
2500 0.9978331527627302
2501 0.9989165763813651
2502 0.9989165763813651
2503 0.9989165763813651
2504 0.9967497291440953
2505 0.9989165763813651
2506 0.9989165763813651
2507 0.9978331527627302
2508 0.9989165763813651
2509 0.9989165763813651
2510 0.9967497291440953
2511 0.9945828819068255
2512 0.9989165763813651
2513 0.9989165763813651
2514 0.9989165763813651
2515 0.9956663055254604
2516 0.9989165763813651
2517 0.997833152

2824 0.9989165763813651
2825 0.9989165763813651
2826 0.9989165763813651
2827 0.9989165763813651
2828 0.9989165763813651
2829 0.9989165763813651
2830 0.9989165763813651
2831 0.9989165763813651
2832 0.9989165763813651
2833 0.9989165763813651
2834 0.9989165763813651
2835 0.9978331527627302
2836 1.0
2837 0.9967497291440953
2838 0.9989165763813651
2839 0.9989165763813651
2840 0.9989165763813651
2841 0.9989165763813651
2842 0.9989165763813651
2843 0.9989165763813651
2844 0.9989165763813651
2845 0.9989165763813651
2846 0.9989165763813651
2847 0.9989165763813651
2848 0.9989165763813651
2849 0.9989165763813651
2850 0.9989165763813651
2851 0.9989165763813651
2852 0.9989165763813651
2853 0.9989165763813651
2854 0.9989165763813651
2855 0.9989165763813651
2856 1.0
2857 0.9989165763813651
2858 0.9989165763813651
2859 1.0
2860 0.9989165763813651
2861 0.9989165763813651
2862 0.9989165763813651
2863 0.9989165763813651
2864 0.9989165763813651
2865 0.9945828819068255
2866 0.9989165763813651
2867 0.998916

3253 1.0
3254 1.0
3255 1.0
3256 0.9989165763813651
3257 1.0
3258 1.0
3259 1.0
3260 1.0
3261 1.0
3262 1.0
3263 1.0
3264 1.0
3265 1.0
3266 1.0
3267 1.0
3268 1.0
3269 1.0
3270 1.0
3271 0.9989165763813651
3272 1.0
3273 1.0
3274 1.0
3275 1.0
3276 1.0
3277 1.0
3278 1.0
3279 1.0
3280 1.0
3281 1.0
3282 1.0
3283 1.0
3284 1.0
3285 1.0
3286 0.9989165763813651
3287 1.0
3288 1.0
3289 0.9989165763813651
3290 1.0
3291 1.0
3292 1.0
3293 1.0
3294 0.9989165763813651
3295 1.0
3296 1.0
3297 1.0
3298 1.0
3299 1.0
3300 1.0
3301 1.0
3302 0.9989165763813651
3303 1.0
3304 1.0
3305 1.0
3306 1.0
3307 1.0
3308 1.0
3309 1.0
3310 1.0
3311 1.0
3312 1.0
3313 1.0
3314 1.0
3315 1.0
3316 1.0
3317 1.0
3318 1.0
3319 1.0
3320 1.0
3321 1.0
3322 1.0
3323 1.0
3324 1.0
3325 1.0
3326 1.0
3327 0.9989165763813651
3328 1.0
3329 1.0
3330 1.0
3331 1.0
3332 0.9989165763813651
3333 1.0
3334 0.9989165763813651
3335 0.9989165763813651
3336 0.9989165763813651
3337 0.9989165763813651
3338 1.0
3339 1.0
3340 1.0
3341 1.0
3342 1.0
3343 1.0
3